In [1]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
from Diffusion import DiffusionModel
from utils import *
import wandb

In [2]:
img_size = 32
batch_size = 4
epochs = 300
learning_rate = 0.0001
log = True
noise_steps = 2000

In [3]:
transform = transforms.Compose([
            transforms.Resize(img_size + int(.25*img_size)),     # Scale up by 25% to enable random crop
            transforms.RandomResizedCrop(img_size, scale=(0.8, 1.0)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # normalize mean and std for n=3 color channels
        ])

In [4]:
training_set = torchvision.datasets.CIFAR10('./data', train=True, transform=transform, download=True)
testing_set = torchvision.datasets.CIFAR10('./data', train=False, transform=transform, download=True)
training_set, validation_set = torch.utils.data.random_split(training_set, [45000, 5000])

Files already downloaded and verified
Files already downloaded and verified


In [5]:
training_loader = torch.utils.data.DataLoader(training_set, batch_size=batch_size, shuffle=True, num_workers=10,  pin_memory=True)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=batch_size, shuffle=True, num_workers=10, pin_memory=True)
testing_loader = torch.utils.data.DataLoader(testing_set, batch_size=batch_size, shuffle=True, num_workers=10, pin_memory=True)

In [6]:
model = DiffusionModel(
    learning_rate=learning_rate,
    in_channels=3,
    out_channels=3,
    noise_steps=noise_steps,
    beta_start=1e-4,
    beta_end=0.02,
    img_size=img_size,
    device="cuda",
    num_class=10
)

In [7]:
if log:
    wandb.init(
        project="DenoiseDiffusion",
        config={
        "dataset": "CIFAR-100",
        "img_size": img_size,
        "learning_rate": learning_rate,
        "batch_size": batch_size,
        "epochs": epochs,
        "Type": "DDPM",
        "Save": "Save-0"
        }
    )

model.train(training_loader, validation_loader, epochs, log=log)
wandb.finish()

wandb: Currently logged in as: woodleighj (jackwoodleigh). Use `wandb login --relogin` to force relogin


Epoch 0... 


100%|██████████████████████████████████████████████████████████████████████████████| 1250/1250 [00:49<00:00, 25.31it/s]


Average Training Loss = 0.037601285812258724, Average Validation Loss = 0.02464626275151968
Model saved to diffusion_model.pth

Epoch 1... 


100%|██████████████████████████████████████████████████████████████████████████████| 1250/1250 [00:57<00:00, 21.73it/s]


...

Average Training Loss = 0.01830212311156922, Average Validation Loss = 0.018751077740453183
Model saved to diffusion_model.pth

Epoch 13... 


100%|██████████████████████████████████████████████████████████████████████████████| 1250/1250 [01:00<00:00, 20.50it/s]


Average Training Loss = 0.018231457365324926, Average Validation Loss = 0.018592383346147837
Model saved to diffusion_model.pth

Epoch 14... 


 54%|█████████████████████████████████████████▊                                   | 6102/11250 [09:27<07:58, 10.75it/s]


KeyboardInterrupt: 